# Analizy Przestrzenne 2022Z
# Projekt 1: Wskazanie optymalnej lokalizacji inwestycji hotelowej – analizy wielokryterialne
dane wejściowe: 
* BDOT caly
* NMT sklejony dla gminy
* gazociag
* dane glebowo-rolnicze (gleby)
* granice gminy

dane wyjściowe:
* TBA

# Import bibliotek, ustawienie środowiska, ustawienie ścieżek do warstw

In [40]:
import sys
import arcpy
import os

In [41]:
# sprawdzanie rozszerzeń
# https://pro.arcgis.com/en/pro-app/latest/arcpy/functions/checkextension.htm
arcpy.CheckOutExtension("Spatial")
arcpy.CheckOutExtension("3D")

'CheckedOut'

In [42]:
# nadpisywanie plików - nie działa, bo ESRI...
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

In [43]:
nazwa_gminy = "Świeradów-Zdrój"

# workspace_path = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\MyProject1.gdb"  # gdb
# folder_bdot = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\DANE\bdot_caly"
# path_nmt = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\DANE\nmt_gminy\nmt_swieradow.tif"
# path_gazociag = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\DANE\gazociag\gazociag.shp"
# path_gleby = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\DANE\gleby\gleby_gm_swieradow.shp"
# path_granice_gminy = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\DANE\granice_gminy\gmina_swieradow.gpkg\main.gmina_swieradow"
workspace_path = r"C:\SEM5\AP\p1\analiza_hotele\analiza_hotele.gdb"  # gdb
folder_bdot = r"C:\SEM5\AP\p1\DANE\bdot_caly"
path_nmt = r"C:\SEM5\AP\p1\DANE\nmt_gminy\nmt_swieradow.tif"
path_gazociag = r"C:\SEM5\AP\p1\DANE\gazociag\gazociag.shp"
path_gleby = r"C:\SEM5\AP\p1\DANE\gleby\gleby_gm_swieradow.shp"
path_granice_gminy = r"C:\SEM5\AP\p1\DANE\granice_gminy\gmina_swieradow.gpkg\main.gmina_swieradow"
bufor_gminy = arcpy.analysis.Buffer(path_granice_gminy, "bufor_gminy", 100)


In [44]:
# ustawienia środowiska pracy
# data paths
arcpy.env.workspace = workspace_path  # geodatabase
# processing extent
arcpy.env.extent = str(bufor_gminy)  # path to bufor_gminy
# output coordinate system
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("ETRS 1989 Poland CS92")
# cell size
arcpy.env.cellSize = 1
# nadpisywanie plików
arcpy.env.overwriteOutput = True

# Ogarnięcie nazw plików z folderu BDOT i zapisanie potrzebnych ścieżek

In [45]:
# Pozbycie się kropek z nazw plików w folderze bdot_caly

bdot = dict()
names = {'budynki':'bubd_a', 'drogi':'skdr_l', 'rzeki':'swrs_l', 'jeziora':'ptwp_a', 'lasy':'ptlz_a'}
extensions = ('.shp', '.prj', '.xlsx', '.dbf', '.shx', '.cpg')
for filename in os.listdir(folder_bdot):
    if os.path.isfile(os.path.join(folder_bdot, filename)):
        src = rf"{folder_bdot}\{filename}"
        if src.endswith(extensions):
            if src.count('.') == 4:
                for k, v in names.items():
                    if v in filename.lower():
                        dst = src.replace('.', '', 3)
                        os.rename(src, dst)
                        if src.endswith('.shp'):
                            bdot[k] = dst

In [46]:
# Kreacja słownika bdot jesli juz mamy dobre nazwy warstw

bdot = dict()
names = {'budynki':'bubd_a', 'drogi':'skdr_l', 'rzeki':'swrs_l', 'jeziora':'ptwp_a', 'lasy':'ptlz_a'}
extensions = ('.shp', '.prj', '.xlsx', '.dbf', '.shx', '.cpg')
for filename in os.listdir(folder_bdot):
    if os.path.isfile(os.path.join(folder_bdot, filename)):
        src = rf"{folder_bdot}\{filename}"
        if src.endswith(extensions):
            if src.count('.') == 1:
                for key, name in names.items():
                    if name in filename.lower() and src.endswith('.shp'):
                        bdot[key] = src

In [47]:
# bdot = {'budynki': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_BUBD_A.shp', 'lasy': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_PTLZ_A.shp', 'jeziora': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_PTWP_A.shp', 'drogi': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_SKDR_L.shp', 'rzeki': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_SWRS_L.shp'}
bdot

{'budynki': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_BUBD_A.shp', 'lasy': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_PTLZ_A.shp', 'jeziora': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_PTWP_A.shp', 'drogi': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_SKDR_L.shp', 'rzeki': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_SWRS_L.shp'}

# Kryterium 1 - odległość od konkurencji minimum 400m

In [48]:
query = "FUNSZCZ LIKE '%hotel%' OR FUNSZCZ LIKE '%motel%' OR FUNSZCZ LIKE '%pensjonat%' OR FUNSZCZ LIKE '%domWypoczynkowy%'"
hotele = arcpy.analysis.Select(bdot['budynki'], "hotele", query)
ed = arcpy.sa.EucDistance(hotele)
minimum = 400 # 400
maximum = 800 # 800

k1 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))

# Kryterium 2 - odległość od budynków mieszkalnych od 25 do 150m

In [49]:
query = "FUNOGBUD LIKE '%miesz%' OR FUNOGBUD LIKE '%Miesz%'"  # 1036
mieszkalne = arcpy.analysis.Select(bdot['budynki'], "mieszkalne", query)
ed = arcpy.sa.EucDistance(mieszkalne)
minimum = 25 # 25
maximum = 50 # 50
fuzzy1 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))
minimum = 150 # 150
maximum = 125 # 125
fuzzy2 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))

k2 = arcpy.sa.Times(fuzzy1, fuzzy2)

# Kryterium 3 - odległość od istniejących dróg od 15 do 100m

In [50]:
drogi = bdot['drogi']
ed = arcpy.sa.EucDistance(drogi)
minimum = 15 # 15
maximum = 20 # 20
fuzzy1 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))
minimum = 200 # 200
maximum = 100 # 100
fuzzy2 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))

k3 = arcpy.sa.Times(fuzzy1, fuzzy2)

# Kryterium 4 - odległość od rzek i zbiorników wodnych nieprzekraczalna strefa ochronna poniżej 20m

In [51]:
query = "X_KOD='SWRS01' OR X_KOD='SWRS02'"
rzeki = arcpy.analysis.Select(bdot['rzeki'], 'rzeki', query)
bufor_rzeki = arcpy.analysis.Buffer(rzeki, 'bufor_rzeki', 0.5)
jeziora = bdot['jeziora']
wody = arcpy.management.Merge([bufor_rzeki, jeziora], 'wody')
ed = arcpy.sa.EucDistance(wody)
minimum = 20 # 20
maximum = 25 # 25
fuzzy1 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))
minimum = 1000 # 1000
maximum = 100 # 100
fuzzy2 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))

k4 = arcpy.sa.Times(fuzzy1, fuzzy2)

# Kryterium 5 - pokrycie terenu nie w lesie

In [52]:
lasy_raster = arcpy.conversion.FeatureToRaster(bdot['lasy'], "RODZAJ", 'lasy_raster')

k5 = arcpy.sa.Reclassify(lasy_raster, "RODZAJ", arcpy.sa.RemapValue([["las", 0], ["zadrzewienie", 1],["zagajnik", 0], ["NODATA", 1]]))

# Kryterium 6 - nachylenie stoków maksymalnie 20%

In [53]:
slope_perc = arcpy.sa.Slope(path_nmt, "PERCENT_RISE", 1, "PLANAR", "METER")
minimum = 20 # 20
maximum = 5 # 5

k6 = arcpy.sa.FuzzyMembership(slope_perc, arcpy.sa.FuzzyLinear(minimum, maximum))

# Kryterium 7 - dostęp światła słonecznego stoki południowe (SW-SE)

In [54]:
aspect = arcpy.sa.Aspect(path_nmt)
minimum = 110 # 110
maximum = 155 # 155
fuzzy1 = arcpy.sa.FuzzyMembership(aspect, arcpy.sa.FuzzyLinear(minimum, maximum))
minimum = 250 # 250 
maximum = 205 # 205
fuzzy2 = arcpy.sa.FuzzyMembership(aspect, arcpy.sa.FuzzyLinear(minimum, maximum))

k7 = arcpy.sa.Times(fuzzy1, fuzzy2)
k7 = arcpy.sa.Con(k7, 1, k7, "VALUE = -1")

# Kryterium 8 - poza strefą ochronną gazociągu minimum 25m

In [55]:
ed = arcpy.sa.EucDistance(path_gazociag)
minimum = 25 # 25
maximum = 26 # 26
fuzzy1 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))
minimum = 1000 # 1000
maximum = 100  # 100
fuzzy2 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))

k8 = arcpy.sa.Times(fuzzy1, fuzzy2)

# Kryterium 9 - użytkowanie terenu - przydatne gleby

In [56]:
codeblock = """
def is_useful(string):
    if string in ['G','E','M','T','F','Fb','Fc','FG']:
        return 0
    return 1
"""
expression = "is_useful(!TYP!)"
field_name = "przydatne"
gleby = arcpy.management.CalculateField(path_gleby, field_name, expression,"PYTHON3", codeblock)

k9 = arcpy.conversion.FeatureToRaster(gleby, "przydatne", "k9")

# Kryterium 10 - przydatne działki obszary 1-6ha

# Kryterium 11 - kształt obszaru jak najbardziej zwarty

# Łączenie kryteriów

# TRASH

In [57]:
# ?arcpy.analysis.Select

In [58]:
# arcpy.Describe(bufor_gminy)

In [59]:
# # TYLKO RAZ - pozbycie się kropek z nazw plików w folderze bdot_caly
# bdot = dict()
# extensions = ('.shp', '.prj', '.xlsx', '.dbf', '.shx', '.cpg')
# for filename in os.listdir(folder_bdot):
#     if os.path.isfile(os.path.join(folder_bdot, filename)):
#         src = rf"{folder_bdot}\{filename}"
#         if src.endswith(extensions):
#             if src.count('.') == 4:
#                 if "bubd_a" in filename.lower():
#                     dst = src.replace('.', '', 3)
#                     os.rename(src, dst)
#                     if src.endswith('.shp'):
#                         bdot['budynki'] = dst
#                 if "skdr_l" in filename.lower():
#                     dst = src.replace('.', '', 3)
#                     os.rename(src, dst)
#                     if src.endswith('.shp'):
#                         bdot['drogi'] = dst
#                 if "swrs_l" in filename.lower():
#                     dst = src.replace('.', '', 3)
#                     os.rename(src, dst)
#                     if src.endswith('.shp'):
#                         bdot['rzeki'] = dst
#                 if "ptwp_a" in filename.lower():
#                     dst = src.replace('.', '', 3)
#                     os.rename(src, dst)
#                     if src.endswith('.shp'):
#                         bdot['jeziora'] = dst
#                 if "ptlz_a" in filename.lower():
#                     dst = src.replace('.', '', 3)
#                     os.rename(src, dst)
#                     if src.endswith('.shp'):
#                         bdot['lasy'] = dst